# Use Case - Granule Search (FedEO)

This chapter provides a comprehensive and detailed process about how to implement a CEOS
OpenSearch client, which includes how to retrieve the IDN collection ID for the collection of
interest, and how to build an OpenSearch request.

In [16]:
import re
import json, requests, xml
import pandas as pd

from xml.dom import minidom
from xml.etree import ElementTree
from IPython.display import Markdown as md

In [ ]:
def get_api_request(template, os_querystring):
  #
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and 
  # return as short HTTP URL without empty parameters.
  #
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  # print("API request: " + template)
            
  return (template)

## FedEO Systems

There are two operational FedEO systems to which end-users have access.

- FedEO PROD – this is FedEO production instance and is available to all users.  Location: https://fedeo.ceos.org/opensearch/description.xml

- FedEO TEST – this is FedEO testing instance used by data partners and external clients to perform testing before changes are made to the FedEO production instance. Location: https://geo.spacebel.be/opensearch/description.xml

The production instance will provide access to collections verified to be
at least minimally compliant.  The collections registered in IDN are a subset of these.  The TEST instance may provide access to additional collections (e.g.,
new collections undergoing testing and not yet registered in the IDN), and capabilities which have not yet been tested sufficiently to move to the production system.

## Retrieve Collection OSDD via IDN OpenSearch

The steps in this section are identical to the steps explained in section 6.2. To fit the end-to-end
FedEO use case, the parameter values for the collection search has been selected in such a way
that the granule search is eventually performed through FedEO.


| **Use Case Overview**  |  | 
| -------- | --------- | 
| Title | Clients start from the IDN OpenSearch | 
| Description | This use case describes steps for retrieving a collection ID from the IDN OpenSearch. | 
| Actors | OpenSearch client, IDN OpenSearch Server | 
| Initial Status and Preconditions | Clients start from the IDN OpenSearch | 

The following steps describe this use case.

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [18]:
url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

In [19]:
md("The OpenSearch Description Document is accessible at the fixed location [{url}]({url}) and contains the URL template to be used for collection search.".format(url=url_osdd))

The OpenSearch Description Document is accessible at the fixed location [https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc](https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc) and contains the URL template to be used for collection search.

In [20]:
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
# md("```xml\n" + xmlstr + "\n```\n")

The OSDD may contain URL templates for multiple values for `type` (media types) and multiple values for `rel` (relations).  The `rel` values have the following meaning (if present) as defined in [[RD1]](#RD1). :

| **rel** |  **description**  | 
| ----- | ----- |
| collection |  URL template to be used for collection search.  |
| results |  URL template to be used for granule search (default).  |
| service |  URL template to be used for service or application search.  |

Find the URL template for collection search with Atom responses.

In [21]:
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters (e.g. spatial footprint, temporal extent and keyword). 

An example request can be formed as follows.

In [22]:
request_url = get_api_request(collection_template, {'count': '10', 'searchTerms': 'C1532648141-ESA'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&numberOfResults=10&clientId=ceosOpenSearchDoc'

In [23]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-05T12:40:42.588Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; C1532648141-ESA</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="C1532648141-ESA" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>1</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1532648141-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">CryoSat products</title>      
          <summary type="text">CryoSat's primary payload is the SAR/Interferometric Radar Altimeter (SIRAL) (https://earth.esa.int/eogateway/instruments/siral) which has extended capabilities to meet the measurement requirements for ice-sheet elevation and sea-ice freeboard.  CryoSat also carries three star trackers for measuring the orientation of the baseline. In addition, a radio receiver called Doppler Orbit and Radio Positioning Integration by Satellite (DORIS) and a small laser retroreflector ensures that CryoSat's position will be accurately tracked.  More detailed information on CryoSat instruments is available on the CryoSat mission page.  The following CryoSat datasets are available and distributed to registered users:  Level 1B and L2 Ice products: FDM, LRM, SAR and SARIn Consolidated Level 2 (GDR): (LRM+SAR+SARIN) consolidated ice products over an orbit Intermediate Level 2 Ice products: LRM, SAR and SARIn L1b and L2 Ocean Products: GOP and IOP CryoTEMPO EOLIS Point Products CryoTEMPO EOLIS Gridded Products  Detailed information concerning each of the above datasets is available in the CryoSat Products Overview (https://earth.esa.int/eogateway/missions/cryosat/products) and in the news item: CryoSat Ocean Products now open to scientific community (https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/cryosat/news/-/asset_publisher/47bD/content/cryosat-ocean-products-now-open-to-scientific-community).  CryoSat Level 1B altimetric products contain time and geo-location information as well as SIRAL measurements in engineering units. Calibration corrections are included and have been applied to the window delay computations. In Offline products, geophysical corrections are computed from Analysis Auxiliary Data Files (ADFs), whereas in FDM products corrections are computed for Forecast ADFs. All corrections are included in the data products and therefore the range can be calculated by taking into account the surface type.  The Offline Level 2 LRM, SAR and SARIn ice altimetric products are generated 30 days after data acquisition and are principally dedicated to glaciologists working on sea-ice and land-ice areas. The Level 2 FDM products are near-real time ocean products, generated 2-3 hours after data acquisition, and fulfill the needs of some ocean operational services. Level 2 products contain the time of measurement, the geo-location and the height of the surface.  IOP and GOP are outputs of the CryoSat Ocean Processor. These products are dedicated to the study of ocean surfaces, and provided specifically for the needs of the oceanographic community. IOP are generated 2-3 days after data sensing acquisition and use the DORIS Preliminary Orbit. GOP are typically generated 30 days after data sensing acquisition and use the DORIS Precise Orbit. Geophysical corrections are computed from the Analysis ADFs, however following the oceanographic convention the corrections are available but not directly applied to the range (as for FDM).  The CryoSat ThEMatic PrOducts (Cryo-TEMPO) projects aim to deliver a new  paradigm of simplified, harmonized, and agile CryoSat-2 products, that are  easily accessible to new communities of non-altimeter experts and end users. The Cryo-TEMPO datasets include dedicated products over five thematic areas, covering Sea Ice, Land Ice, Polar Ocean, Coastal Ocean and Inland Water, together with a novel SWATH product (CryoTEMPO-EOLIS) that exploits CryoSat's SARIn mode over ice sheet margins. The standard Cryo-TEMPO products include fully-traceable uncertainties and use rapidly evolving, state-of-the-art processing dedicated to each thematic area. Throughout the project, the  products will be constantly evolved, and validated by a group of Thematic Users, thus ensuring optimal relevance and impact for the intended target  communities. More information on the Cryo-TEMPO products can be found on the Project Website (http://cryosat.mssl.ucl.ac.uk/tempo/index.html). The CryoTEMPO-EOLIS swath product exploits CryoSat's SARIn mode and the novel Swath processing technique to deliver increased spatial and temporal coverage of time-dependent elevation over land ice, a critical metric for tracking ice mass trends in support to a wide variety of end-users. The CryoTEMPO-EOLIS swath product exploits CryoSat's SARIn mode and the novel Swath processing technique to deliver increased spatial and temporal coverage of time-dependent elevation over land ice, a critical metric for tracking ice mass trends in support to a wide variety of end-users.The dataset consists of systematic reprocessing of the entire CryoSat archive to generate new L2-Swath products, increasing data sampling by 1 to 2 orders of magnitude compared with the operational L2 ESA product. In addition, the EOLIS dataset is joined with the ESA L2 Point-Of-Closest-Approach to generate monthly DEM (Digital Elevation Model) products. This dataset will further the ability of the community to analyse and understand trends across the Greenland Ice Sheet margin, Antarctica and several mountain glaciers and ice caps around the world.</summary>      
          <updated/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=CryoSat.products" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=CryoSat.products&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://doi.org/10.5270/CR2-120cf4c" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-cbow23i" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-05fbc11" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-41ad749" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-2cnblvi" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-fbae3cd" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-u3805kw" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-6afef01" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-k1o4pyh" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-614f341" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-53hztdl" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-388fb81" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-gsyvnx0" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-65cff05" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-he6wfkr" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-b0fffc1" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-2xs4q4l" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-11aea35" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-656eeb1" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-7ece675" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-3205d1e" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-e2dd631" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-1d3c78c" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-52fafc4" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-3f3c491" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-996c491" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-b39c015" hreflang="en-US" rel="via"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?CryoSat.products_NA" hreflang="en-US" rel="enclosure" title="CryoSat.products" type="text/html"/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1532648141-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1532648141-ESA</dc:identifier>      
          <dc:date>2010-04-08T00:00:00.000Z/</dc:date>      
          <echo:datasetId>CryoSat products</echo:datasetId>      
          <echo:shortName>CryoSat.products</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>CryoSat.products_NA</echo:entryId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>0.5</relevance:score>      
          <echo:tag>         
               <echo:tagKey>int.esa.fedeo</echo:tagKey>         
          </echo:tag>      
          <echo:is_fedeo>true</echo:is_fedeo>      
     </entry>   
</feed>
```


**Step 3**  
>  From the IDN OpenSearch response, obtain the OSDD endpoint for the collection by parsing the href attribute under `<link rel="search" type=”application/opensearchdescription+xml” >` Note that this collection happens to be accessed via FedEO. 

In [24]:
root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml" />
    
```


## Retrieve Granules via OpenSearch

After retrieving the OSDD endpoint from the IDN by querying through the CEOS OpenSearch API, OpenSearch clients will sequentially interact with the host remote server for inventory search.
The following table shows the basic information about the use case of interacting with FedEO via OpenSearch.


| **Use Case Overview**  |  | 
| -------- | --------- | 
| Title | Interact with FedEO server for inventory search | 
| Description | This use case describes steps for executing inventory search based on the IDN collection OSDD of interest. | 
| Actors | OpenSearch client, FedEO Server | 
| Initial Status and Preconditions | Clients have the FedEO OpenSearch URL from IDN. | 

**Step 4**  
>  Obtain the collection specific OSDD from the IDN response.  

In [25]:
url_osdd = el.attrib['href']
el.attrib['href']

'https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products'

Retrieve the OSDD document

In [26]:
response = requests.get( el.attrib['href'] )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
md("```xml\n" + xmlstr + "\n```\n")


```xml
<?xml version="1.0" ?><OpenSearchDescription xmlns="http://a9.com/-/spec/opensearch/1.1/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/" xmlns:param="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/" xmlns:semantic="http://a9.com/-/opensearch/extensions/semantic/1.0/" xmlns:sru="http://a9.com/-/opensearch/extensions/sru/2.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/" xmlns:wsse="http://docs.oasis-open.org/wss/2004/01/oasis-200401-wss-wssecurity-secext-1.0.xsd">  
  <ShortName>FedEO</ShortName>  
  <Description>Provides interoperable access, following ISO/OGC interface guidelines, to Earth Observation metadata.</Description>  
  <Tags>FedEO Clearinghouse, ESA, Earth Observation, Digital Repository, HMA, HMA-S, HMA-SE, CEOS-OS-BP-V1.1/L1.</Tags>  
  <Url rel="self" template="https://fedeo.ceos.org/collections/series/items/CryoSat.products/api?parentIdentifier=CryoSat.products" type="application/opensearchdescription+xml"/>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fogcapi-features-1%2F1.0%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3D%22https%3A%2F%2Fstacspec.org%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson%3Bprofile%3Dhttps%3A%2F%2Fstacspec.org&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/geo+json;profile=https://stacspec.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fld%2Bjson%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/ld+json;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Frdf%2Bxml%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="application/rdf+xml;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fhtml&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/html" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22http%3A%2F%2Fdata.europa.eu%2F930%2F%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22http%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3D%22https%3A%2F%2Fschema.org%22&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=&quot;https://schema.org&quot;" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttp%3A%2F%2Fdata.europa.eu%2F930%2F&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=http://data.europa.eu/930/" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttp%3A%2F%2Fwww.opengis.net%2Fspec%2Fos-geojson%2F1.0&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=http://www.opengis.net/spec/os-geojson/1.0" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=text%2Fturtle%3Bprofile%3Dhttps%3A%2F%2Fschema.org&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;source={referrer:source?}" type="text/turtle;profile=https://schema.org" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}&amp;recordSchema={sru:recordSchema?}" type="application/atom+xml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		    <param:Parameter name="recordSchema" value="{sru:recordSchema}">      
			      <param:Option label="O&amp;M 1.1" value="om"/>      
			      <param:Option label="O&amp;M 1.0" value="om10"/>      
			      <param:Option label="server-choice" value="server-choice"/>      
		    </param:Parameter>    
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Url indexOffset="1" pageOffset="1" rel="results" template="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fgeo%2Bjson&amp;parentIdentifier=CryoSat.products&amp;startRecord={startIndex?}&amp;startPage={startPage?}&amp;limit={count?}&amp;uid={geo:uid?}&amp;acquisitionType={eo:acquisitionType?}&amp;bbox={geo:box?}&amp;name={geo:name?}&amp;lat={geo:lat?}&amp;lon={geo:lon?}&amp;radius={geo:radius?}&amp;geometry={geo:geometry?}&amp;productVersion={eo:productVersion?}&amp;productionStatus={eo:productionStatus?}&amp;completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&amp;acquisitionStation={eo:acquisitionStation?}&amp;processingDate={eo:processingDate?}&amp;instrument={eo:instrument?}&amp;platform={eo:platform?}&amp;orbitNumber={eo:orbitNumber?}&amp;startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&amp;productType={eo:productType?}&amp;platformSerialIdentifier={eo:platformSerialIdentifier?}&amp;processingCenter={eo:processingCenter?}&amp;sensorMode={eo:sensorMode?}&amp;orbitDirection={eo:orbitDirection?}&amp;facetLimit={sru:facetLimit?}&amp;sortKeys={sru:sortKeys?}&amp;datetime={time:start?}/{time:end?}" type="application/geo+json" xmlns:os="http://a9.com/-/spec/opensearch/1.1/">    		
		
		
		
		
		    <param:Parameter minInclusive="2010-07-15T00:00:00.000Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:start}"/>    
		    <param:Parameter maxInclusive="2024-11-02T23:59:59.999Z" pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}(T[0-9]{2}:[0-9]{2}:[0-9]{2}(\\.[0-9]{0,3})?(Z)$)?" value="{time:end}"/>    
		
		    <param:Parameter name="acquisitionType" value="{eo:acquisitionType}">      
					
			      <param:Option label="NOMINAL" value="NOMINAL"/>      		
					
		    </param:Parameter>    
		
		    <param:Parameter name="completionTimeFromAscendingNode" pattern="\[[0-9]+,[0-9]+\]|\[?[0-9]+|[0-9]+\]?|\{[0-9]+,[0-9]+\}" value="{eo:completionTimeFromAscendingNode}"/>    
		
		
		
		
		
		
	
		
		
		    <param:Parameter name="orbitDirection" value="{eo:orbitDirection}">      
			      <param:Option label="ASCENDING" value="ASCENDING"/>      		
			      <param:Option label="DESCENDING" value="DESCENDING"/>      		
		    </param:Parameter>    
		
		    <param:Parameter name="orbitNumber" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:orbitNumber}"/>    
		
		    <param:Parameter name="productionStatus" value="{eo:productionStatus}">      
			
					
					
					
					
					
			      <param:Option label="Data acquisition archived" value="ARCHIVED"/>      				
		    </param:Parameter>    
		
		
		
		
					
		    <param:Parameter name="startTimeFromAscendingNode" pattern="(\[|\])[0-9]+,[0-9]+(\[|\])|(\[|\])?[0-9]+|[0-9]+(\[|\])?|\{[0-9]+,[0-9]+\}" value="{eo:startTimeFromAscendingNode}"/>    
		
		    <param:Parameter maxInclusive="90" minInclusive="-90" name="lat" value="{geo:lat}"/>    
		    <param:Parameter maxInclusive="180" minInclusive="-180" name="lon" value="{geo:lon}"/>    
		
		
		
	    <param:Parameter name="productVersion" value="{eo:productVersion}">      
      <param:Option label="c001" value="c001"/>      
      <param:Option label="c002" value="c002"/>      
      <param:Option label="c003" value="c003"/>      
      <param:Option label="c004" value="c004"/>      
      <param:Option label="d001" value="d001"/>      
      <param:Option label="d002" value="d002"/>      
      <param:Option label="d003" value="d003"/>      
      <param:Option label="e001" value="e001"/>      
      <param:Option label="e002" value="e002"/>      
      <param:Option label="e003" value="e003"/>      
      <param:Option label="e004" value="e004"/>      
      <param:Option label="e005" value="e005"/>      
    </param:Parameter>    
    <param:Parameter name="acquisitionStation" value="{eo:acquisitionStation}">      
      <param:Option label="KS" value="KS"/>      
    </param:Parameter>    
    <param:Parameter name="instrument" value="{eo:instrument}">      
      <param:Option label="SIRAL" value="SIRAL"/>      
    </param:Parameter>    
    <param:Parameter name="platform" value="{eo:platform}">      
      <param:Option label="Cryosat" value="Cryosat"/>      
    </param:Parameter>    
    <param:Parameter name="productType" value="{eo:productType}">      
      <param:Option label="SIR1SAR_FR" value="SIR1SAR_FR"/>      
      <param:Option label="SIR_GDR_2_" value="SIR_GDR_2_"/>      
      <param:Option label="SIR_GOPM1B" value="SIR_GOPM1B"/>      
      <param:Option label="SIR_GOPM_2" value="SIR_GOPM_2"/>      
      <param:Option label="SIR_GOPN1B" value="SIR_GOPN1B"/>      
      <param:Option label="SIR_GOPN_2" value="SIR_GOPN_2"/>      
      <param:Option label="SIR_GOPR1B" value="SIR_GOPR1B"/>      
      <param:Option label="SIR_GOPR_2" value="SIR_GOPR_2"/>      
      <param:Option label="SIR_GOP_2_" value="SIR_GOP_2_"/>      
      <param:Option label="SIR_LRMI2_" value="SIR_LRMI2_"/>      
      <param:Option label="SIR_LRM_1B" value="SIR_LRM_1B"/>      
      <param:Option label="SIR_LRM_2_" value="SIR_LRM_2_"/>      
      <param:Option label="SIR_SARI2_" value="SIR_SARI2_"/>      
      <param:Option label="SIR_SAR_1B" value="SIR_SAR_1B"/>      
      <param:Option label="SIR_SAR_2_" value="SIR_SAR_2_"/>      
      <param:Option label="SIR_SINI2_" value="SIR_SINI2_"/>      
      <param:Option label="SIR_SIN_1B" value="SIR_SIN_1B"/>      
      <param:Option label="SIR_SIN_2_" value="SIR_SIN_2_"/>      
      <param:Option label="SIR_SIN_FR" value="SIR_SIN_FR"/>      
    </param:Parameter>    
    <param:Parameter name="platformSerialIdentifier" value="{eo:platformSerialIdentifier}">      
      <param:Option label="2" value="2"/>      
    </param:Parameter>    
    <param:Parameter name="processingCenter" value="{eo:processingCenter}">      
      <param:Option label="PDS" value="PDS"/>      
    </param:Parameter>    
    <param:Parameter name="sensorMode" value="{eo:sensorMode}">      
      <param:Option label="GOP" value="GOP"/>      
      <param:Option label="LRM" value="LRM"/>      
      <param:Option label="SAR" value="SAR"/>      
      <param:Option label="SARIN" value="SARIN"/>      
      <param:Option label="SYNERGY" value="SYNERGY"/>      
    </param:Parameter>    

    <param:Parameter minimum="0" name="geometry" title="inventory which has a spatial extent overlapping this geometry" value="{geo:geometry}">      
      <atom:link href="http://www.opengis.net/wkt/POINT" rel="profile" title="This service accepts WKT Point" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/LINESTRING" rel="profile" title="This service accepts WKT LineString" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTIPOINT" rel="profile" title="This service accepts WKT Points" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/MULTILINESTRING" rel="profile" title="This service accepts WKT LineStrings" xmlns:atom="http://www.w3.org/2005/Atom"/>      
      <atom:link href="http://www.opengis.net/wkt/POLYGON" rel="profile" title="This service accepts WKT Polygons" xmlns:atom="http://www.w3.org/2005/Atom"/>      
    </param:Parameter>    
    <param:Parameter name="sortKeys" value="{sru:sortKeys}">      
      <param:Option label="By startDate (time:start) (ascending)." value="start,time,1"/>      
      <param:Option label="By startDate (time:start) (descending)." value="start,time,0"/>      
      <param:Option label="By startDate (time:end) (ascending)." value="end,time,1"/>      
      <param:Option label="By startDate (time:end) (descending)." value="end,time,0"/>      
      <param:Option label="By uid (ascending)." value="uid,geo,1"/>      
      <param:Option label="By uid (descending)." value="uid,geo,0"/>      
      <param:Option label="By acquisitionType (ascending)." value="acquisitionType,eo,1"/>      
      <param:Option label="By acquisitionType (descending)." value="acquisitionType,eo,0"/>      
      <param:Option label="By productVersion (ascending)." value="productVersion,eo,1"/>      
      <param:Option label="By productVersion (descending)." value="productVersion,eo,0"/>      
      <param:Option label="By productionStatus (ascending)." value="productionStatus,eo,1"/>      
      <param:Option label="By productionStatus (descending)." value="productionStatus,eo,0"/>      
      <param:Option label="By completionTimeFromAscendingNode (ascending)." value="completionTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By completionTimeFromAscendingNode (descending)." value="completionTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By acquisitionStation (ascending)." value="acquisitionStation,eo,1"/>      
      <param:Option label="By acquisitionStation (descending)." value="acquisitionStation,eo,0"/>      
      <param:Option label="By processingDate (ascending)." value="processingDate,eo,1"/>      
      <param:Option label="By processingDate (descending)." value="processingDate,eo,0"/>      
      <param:Option label="By instrument (ascending)." value="instrument,eo,1"/>      
      <param:Option label="By instrument (descending)." value="instrument,eo,0"/>      
      <param:Option label="By platform (ascending)." value="platform,eo,1"/>      
      <param:Option label="By platform (descending)." value="platform,eo,0"/>      
      <param:Option label="By orbitNumber (ascending)." value="orbitNumber,eo,1"/>      
      <param:Option label="By orbitNumber (descending)." value="orbitNumber,eo,0"/>      
      <param:Option label="By startTimeFromAscendingNode (ascending)." value="startTimeFromAscendingNode,eo,1"/>      
      <param:Option label="By startTimeFromAscendingNode (descending)." value="startTimeFromAscendingNode,eo,0"/>      
      <param:Option label="By productType (ascending)." value="productType,eo,1"/>      
      <param:Option label="By productType (descending)." value="productType,eo,0"/>      
      <param:Option label="By platformSerialIdentifier (ascending)." value="platformSerialIdentifier,eo,1"/>      
      <param:Option label="By platformSerialIdentifier (descending)." value="platformSerialIdentifier,eo,0"/>      
      <param:Option label="By processingCenter (ascending)." value="processingCenter,eo,1"/>      
      <param:Option label="By processingCenter (descending)." value="processingCenter,eo,0"/>      
      <param:Option label="By sensorMode (ascending)." value="sensorMode,eo,1"/>      
      <param:Option label="By sensorMode (descending)." value="sensorMode,eo,0"/>      
      <param:Option label="By orbitDirection (ascending)." value="orbitDirection,eo,1"/>      
      <param:Option label="By orbitDirection (descending)." value="orbitDirection,eo,0"/>      
    </param:Parameter>    
  </os:Url>  
  <os:Query role="example" time:end="2024-11-02T00:00:00.000Z" time:start="2010-07-15T00:00:00.000Z" xmlns:os="http://a9.com/-/spec/opensearch/1.1/"/>  
  <LongName>Earth Observation Catalogue</LongName>  
  <Image height="64" type="image/png" width="64">https://fedeo.ceos.org/images/esa_favicon.ico</Image>  
  <Image height="16" type="image/vnd.microsoft.icon" width="16">https://fedeo.ceos.org/images/esa_favicon.ico</Image>  
  <Developer>Spacebel s.a.</Developer>  
  <Attribution>Copyright 2017-2020, European Space Agency.</Attribution>  
  <SyndicationRight>open</SyndicationRight>  
  <AdultContent>false</AdultContent>  
  <Language>en-us</Language>  
  <OutputEncoding>UTF-8</OutputEncoding>  
  <InputEncoding>UTF-8</InputEncoding>  
</OpenSearchDescription>
```


**Step 5**  
>  Based on the (FedEO) OSDD, formulate a CEOS OpenSearch request for granules belonging to that collection.  

In [27]:
# Get the OSDD document for granule search
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
# md("```xml\n" + xmlstr + "\n```\n")

Extract the URL template for granule search with Atom response.

In [28]:
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
granule_search_template = root.find('os:Url[@rel="results"][@type="application/atom+xml"]', ns).attrib['template']

# collection_template = collection_url_atom.attrib['template']
granule_search_template

'https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&parentIdentifier=CryoSat.products&startRecord={startIndex?}&startPage={startPage?}&limit={count?}&uid={geo:uid?}&acquisitionType={eo:acquisitionType?}&bbox={geo:box?}&name={geo:name?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&productVersion={eo:productVersion?}&productionStatus={eo:productionStatus?}&completionTimeFromAscendingNode={eo:completionTimeFromAscendingNode?}&acquisitionStation={eo:acquisitionStation?}&processingDate={eo:processingDate?}&instrument={eo:instrument?}&platform={eo:platform?}&orbitNumber={eo:orbitNumber?}&startTimeFromAscendingNode={eo:startTimeFromAscendingNode?}&productType={eo:productType?}&platformSerialIdentifier={eo:platformSerialIdentifier?}&processingCenter={eo:processingCenter?}&sensorMode={eo:sensorMode?}&orbitDirection={eo:orbitDirection?}&facetLimit={sru:facetLimit?}&sortKeys={sru:sortKeys?}&datetime={time:start?}/{time:end?}&record

Example of FedEO OpenSearch request:

In [29]:
request_url = get_api_request(granule_search_template, {'time:start': '2017-01-01T00:00:00Z', 'time:end': '2017-12-31T00:00:00Z', 'startIndex': '1' } )
request_url

'https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&parentIdentifier=CryoSat.products&startRecord=1&datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z'

**Step 6**  
>  Parse the (FedEO) OpenSearch response and extract the identifier for the granule of interest from `<id>` element.  

In [30]:
response = requests.get( request_url )

# xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
# md("```xml\n" + xmlstr + "\n```\n")

root = ElementTree.fromstring(response.text)
# Extract <entry> element 
# el = root.find('{http://www.w3.org/2005/Atom}entry')
xmltxt = ElementTree.tostring(root, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:feed xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:ns0="http://www.w3.org/2005/Atom" xmlns:ns1="http://a9.com/-/spec/opensearch/1.1/" xmlns:ns2="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:ns3="http://a9.com/-/opensearch/extensions/time/1.0/">
<ns0:author>
<ns0:name>FedEO Clearinghouse</ns0:name>
<ns0:email>eohelp@eo.esa.int</ns0:email>
</ns0:author>
<ns0:generator>FedEO Clearinghouse</ns0:generator>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items</ns0:id>
<ns0:rights>Copyright 2016-2024, European Space Agency</ns0:rights>
<ns0:title>FedEO Clearinghouse - Search Response</ns0:title>
<ns0:updated>2024-12-05T12:40:43Z</ns0:updated>
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products/api" rel="search" type="application/opensearchdescription+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;httpAccept=application/geo%2Bjson;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" rel="alternate" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;httpAccept=application/geo%2Bjson;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" rel="alternate" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/os-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;httpAccept=application/atom%2Bxml" rel="alternate" type="application/metalink+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;httpAccept=application/metalink4%2Bxml" rel="alternate" type="application/metalink4+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;startRecord=1&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z" hreflang="en" rel="self" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;startRecord=1" rel="first" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;startRecord=11" rel="next" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items?httpAccept=application%2Fatom%2Bxml&amp;parentIdentifier=CryoSat.products&amp;datetime=2017-01-01T00:00:00Z/2017-12-31T00:00:00Z&amp;startRecord=940361" rel="last" type="application/atom+xml" />
<ns1:totalResults>940362</ns1:totalResults>
<ns1:startIndex>1</ns1:startIndex>
<ns1:itemsPerPage>10</ns1:itemsPerPage>
<ns1:Query role="request" ns2:parentIdentifier="CryoSat.products" ns3:end="2017-12-31T00:00:00Z" ns3:start="2017-01-01T00:00:00Z" ns1:count="10" ns1:startIndex="1" />
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T004724_20170622T005204_E001</dc:identifier>
<dc:date>2017-06-22T00:47:24Z/2017-06-22T00:52:04Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170626T032145_20170626T032251_E001</dc:identifier>
<dc:date>2017-06-26T03:21:45Z/2017-06-26T03:22:51Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T013811_20170622T013845_E001</dc:identifier>
<dc:date>2017-06-22T01:38:11Z/2017-06-22T01:38:45Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170625T134748_20170625T134909_E001</dc:identifier>
<dc:date>2017-06-25T13:47:48Z/2017-06-25T13:49:09Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T014941_20170622T015312_E001</dc:identifier>
<dc:date>2017-06-22T01:49:41Z/2017-06-22T01:53:12Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T062439_20170622T062518_E001</dc:identifier>
<dc:date>2017-06-22T06:24:39Z/2017-06-22T06:25:18Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T030015_20170622T031107_E001</dc:identifier>
<dc:date>2017-06-22T03:00:15Z/2017-06-22T03:11:07Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T045621_20170622T045646_E001</dc:identifier>
<dc:date>2017-06-22T04:56:21Z/2017-06-22T04:56:46Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170626T031622_20170626T031720_E001</dc:identifier>
<dc:date>2017-06-26T03:16:22Z/2017-06-26T03:17:20Z</dc:date>
</ns0:entry>
<ns0:entry>
<ns0:id>https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml</ns0:id>
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/atom%2Bxml" rel="alternate" title="Atom format" type="application/atom+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om" rel="alternate" title="OGC 10-157r4 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.1&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/gml%2Bxml&amp;recordSchema=om10" rel="alternate" title="OGC 10-157r3 metadata" type="application/gml+xml;profile=&quot;http://www.opengis.net/spec/EOMPOM/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;mode=owc" rel="alternate" title="OGC 17-003r2 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/eo-geojson/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products" rel="alternate" title="OGC 17-069r3 metadata" type="application/geo+json;profile=&quot;http://www.opengis.net/spec/ogcapi-features-1/1.0&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/geo%2Bjson;profile=&quot;https://stacspec.org&quot;" rel="alternate" title="STAC metadata" type="application/geo+json;profile=&quot;https://stacspec.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/vnd.iso.19139%2Bxml" rel="alternate" title="ISO 19139 metadata" type="application/vnd.iso.19139+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson" rel="alternate" title="JSON-LD metadata" type="application/ld+json" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=https://schema.org" rel="alternate" title="JSON-LD (schema.org) metadata" type="application/ld+json;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/ld%2Bjson;profile=http://data.europa.eu/930/" rel="alternate" title="JSON-LD (GeoDCAT-AP) metadata" type="application/ld+json;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml" rel="alternate" title="RDF/XML metadata" type="application/rdf+xml" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=https://schema.org" rel="alternate" title="RDF/XML (schema.org) metadata" type="application/rdf+xml;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=application/rdf%2Bxml;profile=http://data.europa.eu/930/" rel="alternate" title="RDF/XML (GeoDCAT-AP) metadata" type="application/rdf+xml;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle" rel="alternate" title="Turtle metadata" type="text/turtle" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=https://schema.org" rel="alternate" title="Turtle (schema.org) metadata" type="text/turtle;profile=&quot;https://schema.org&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/turtle;profile=http://data.europa.eu/930/" rel="alternate" title="Turtle (GeoDCAT-AP) metadata" type="text/turtle;profile=&quot;http://data.europa.eu/930/&quot;" />
<ns0:link href="https://fedeo.ceos.org/collections/datasets/items/CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001?parentIdentifier=CryoSat.products&amp;httpAccept=text/html" rel="alternate" title="HTML" type="text/html" />
<ns0:link href="https://fedeo.ceos.org/collections/series/items/CryoSat.products?httpAccept=application/vnd.iso.19139-2%2Bxml" rel="up" title="ISO 19139-2 metadata" type="application/vnd.iso.19139-2+xml" />
<ns0:link href="https://science-pds.cryosat.esa.int/?do=download&amp;file=Cry0Sat2_data%2FSIR_SIN_L1%2F2017%2F06%2FCS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001.nc" rel="enclosure" title="Download" type="application/x-netcdf" />
<ns0:title>CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001</ns0:title>
<ns0:updated>2023-04-24T00:45:21Z</ns0:updated>
<dc:identifier>CS_LTA__SIR_SIN_1B_20170622T063622_20170622T063644_E001</dc:identifier>
<dc:date>2017-06-22T06:36:22Z/2017-06-22T06:36:44Z</dc:date>
</ns0:entry>
</ns0:feed>
```


## Available Granule Search Parameters

The set of available granule search parameters depends on the actual collection and is not fixed. Please refer to the [OpenAPI description](https://fedeo.ceos.org/api)  for the complete list of search parameters that
may be advertised by a granule search OSDD. The meaning of these parameters is defined in [CEOS-OS-BP],
[OGC 13-026r9](https://docs.ogc.org/is/13-026r9/13-026r9.html) and [OGC 10-032r8](https://portal.ogc.org/files/?artifact_id=56866).